In [30]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import iBioGen

from scipy.stats import entropy
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, cross_val_score

pd.set_option('display.max_columns', 50)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load pi data for each site

In [33]:
pi_dict = {}
for f in glob.glob("Leke-HI-pis/*.pis"):
    site = f.split("/")[1].split(".")[0]
    pi_dict[site] = np.loadtxt(f, delimiter=",")
    print(site, len(pi_dict[site]))

diversified-CO 52
forest-MN 85
diversified-NO 93
simplified-JO 71
diversified-KE 69
simplified-JE 70
forest-MK 88
simplified-TK 48


In [35]:
%%time
prefix = "/media/sdb1/isaac/Islands2030/MESS_sims/"
neutral_simfile = f"{prefix}/sim-neutral-ntaxa300-SIMOUT.csv"
comp_simfile = f"{prefix}/sim-comp-ntaxa300-SIMOUT.csv"
filt_simfile = f"{prefix}/sim-filt-ntaxa300-SIMOUT.csv"

all_sims_params = []
all_sims_locs = []
for model, simfile in zip(["neutral", "competition", "filtering"],
                            [neutral_simfile, comp_simfile, filt_simfile]):
    params_df, loc_df, tre = iBioGen.util.load_local_sims(simfile, nrows=20000)
    print(model, len(params_df))
    params_df["assembly_model"] = model
    all_sims_params.append(params_df)
    all_sims_locs.append(loc_df)

unpruned_sims_params_df = pd.concat(all_sims_params, axis=0, ignore_index=True)
unpruned_sims_locs_df = pd.concat(all_sims_locs, axis=0, ignore_index=True)


neutral 20000
competition 20000
filtering 20000
CPU times: user 4min 27s, sys: 8.65 s, total: 4min 35s
Wall time: 4min 32s


## Prune sims with similar species richness as observed

In [36]:
## Prune out simulations that have low species richness
## This is very coarse
#all_sims_locs_df = all_sims_locs_df[all_sims_params_df["local_S"] < 100]
#all_sims_params_df = all_sims_params_df[all_sims_params_df["local_S"] < 100]

min_S = 50
max_S = 60
all_sims_locs_df = unpruned_sims_locs_df[unpruned_sims_params_df["local_S"].between(min_S, max_S)]
all_sims_params_df = unpruned_sims_params_df[unpruned_sims_params_df["local_S"].between(min_S, max_S)]
len(all_sims_params_df)

5039

## iBioGen model classification raw pi values

In [41]:
%%time
# Empirical data must fit the shape of the simulated pis
def zero_pad(pi_array, data):
    len_diff = data.shape[1]-len(pi_array)
    pi_array = np.pad(np.array(pi_array), (0, len_diff), mode="constant")
    return pi_array

# Pull pis data out of the sim results and format it
pis = all_sims_locs_df.T.apply(lambda y: sorted([x["pi"] for x in y.dropna()], reverse=True))
pis = pd.DataFrame.from_records(pis.values).fillna(0)

y = all_sims_params_df["assembly_model"]
print(len(pis), len(y))

X = pis

#gbc = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10)
gbc = RandomForestClassifier(n_estimators=100, min_samples_leaf=10, n_jobs=-1)
gbc.fit(X, y)



5039 5039
CPU times: user 3.27 s, sys: 305 ms, total: 3.57 s
Wall time: 2.03 s


RandomForestClassifier(min_samples_leaf=10, n_jobs=-1)

In [42]:
raw_pi_res = {}

for site, pi in pi_dict.items():
    print(site, len(pi))
    result = gbc.predict_proba(zero_pad(pi, X).reshape(1, -1))
    raw_pi_res[site] = result[0]
    break

raw_pi_res = pd.DataFrame(raw_pi_res, index=gbc.classes_).T
raw_pi_res

diversified-CO 52


,competition,filtering,neutral
diversified-CO,0.028596,0.168369,0.803034


### iBiogen model classification with scaled pi values

In [43]:
%%time
X = pis.div(np.sum(pis, axis=1), axis=0)

gbc = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10)
gbc.fit(X, y)

scaled_pi_res = {}
for site, pi in pi_dict.items():
    pi = zero_pad(pi, X)/np.sum(pi)
    result = gbc.predict_proba(pi.reshape(1, -1))
    scaled_pi_res[site] = result[0]
    break
scaled_pi_res = pd.DataFrame(scaled_pi_res, index=gbc.classes_).T
scaled_pi_res

CPU times: user 15.8 s, sys: 34.7 ms, total: 15.8 s
Wall time: 15.8 s


,competition,filtering,neutral
diversified-CO,0.018316,0.040775,0.940909


### iBioGen model classification with hill numbers

In [25]:
def hill_numbers(dat):
    hvals = []
    for i in range(4):
        hval = iBioGen.util._generalized_hill_number(abunds=dat,
                                                     order=i)
        if i == 1: hval = np.exp(entropy(dat))
        hvals.append(hval)
    return hvals

hill_dict = {}
for site, pi in pi_dict.items():
    hill_dict[site] = hill_numbers(pi)
hill_dict

{'diversified-CO': [52.000000000000014,
  36.783068148449956,
  29.5968709971414,
  26.19097633509255],
 'forest-MN': [84.99999999999999,
  60.37886232796552,
  50.22409367043723,
  45.228942279801714],
 'diversified-NO': [92.99999999999996,
  70.3732507508443,
  59.70183117022175,
  54.37943771044167],
 'simplified-JO': [70.99999999999997,
  51.72185754633277,
  43.50041967077615,
  39.61346736395563],
 'diversified-KE': [68.99999999999999,
  50.097648744627485,
  40.91239107427376,
  36.183044914433125],
 'simplified-JE': [69.99999999999997,
  53.321242025138986,
  46.015778402567975,
  42.47139753031754],
 'forest-MK': [87.99999999999999,
  67.50729964298549,
  58.68508435307615,
  54.032500326928144],
 'simplified-TK': [48.0,
  37.908643922764,
  32.874667473117135,
  29.74994132684371]}

In [79]:
%%time
X = all_sims_params_df[["local_S", "pi_h1", "pi_h2", "pi_h3"]]

gbc = GradientBoostingClassifier(n_estimators=500, min_samples_leaf=10)
#gbc = RandomForestClassifier(**rf_params)

gbc.fit(X, y)

raw_hill_res = {}
for site, hills in hill_dict.items():
    result = gbc.predict_proba(np.array(hills).reshape(1, -1))
    raw_hill_res[site] = result[0]
raw_hill_res = pd.DataFrame(raw_hill_res, index=gbc.classes_).T
raw_hill_res

CPU times: user 22.3 s, sys: 29.2 ms, total: 22.3 s
Wall time: 22.3 s


/home/isaac/miniconda3/envs/iBioGen/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/isaac/miniconda3/envs/iBioGen/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/isaac/miniconda3/envs/iBioGen/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/isaac/miniconda3/envs/iBioGen/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/isaac/miniconda3/envs/iBioGen/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Grad

,competition,filtering,neutral
diversified-CO,0.113204,0.013553,0.873242
forest-MN,0.981032,0.002276,0.016692
diversified-NO,0.981032,0.002276,0.016692
simplified-JO,0.981032,0.002276,0.016692
diversified-KE,0.981032,0.002276,0.016692
simplified-JE,0.981032,0.002276,0.016692
forest-MK,0.981032,0.002276,0.016692
simplified-TK,0.989503,0.001462,0.009035


## Do all intelligently pruning to reasonable S

In [87]:
%%time
scale_pi = True
S_tol = 10
raw_pi_res = {}

for site, pi in pi_dict.items():
    obs_S = len(pi)
    print(site, obs_S)

    min_S = obs_S - S_tol
    max_S = obs_S + S_tol
    all_sims_locs_df = unpruned_sims_locs_df[unpruned_sims_params_df["local_S"].between(min_S, max_S)]
    all_sims_params_df = unpruned_sims_params_df[unpruned_sims_params_df["local_S"].between(min_S, max_S)]
    print("  nsims", end="\t")
    for model in ["neutral", "filtering", "competition"]:
        print(model, np.sum(all_sims_params_df["assembly_model"] == model), end="\t")
    print("")
    # Pull pis data out of the sim results and format it
    pis = all_sims_locs_df.T.apply(lambda y: sorted([x["pi"] for x in y.dropna()], reverse=True))
    pis = pd.DataFrame.from_records(pis.values).fillna(0)

    y = all_sims_params_df["assembly_model"]
    if scale_pi:
        X = pis.div(np.sum(pis, axis=1), axis=0)
        pi = zero_pad(pi, X)/np.sum(pi)
    else:
        X = pis
        pi = zero_pad(pi, X)
    
    print("  Training ML")
    #gbc = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10)
    gbc = RandomForestClassifier(**rf_params)
    gbc.fit(X, y)

    result = gbc.predict_proba(pi.reshape(1, -1))
    raw_pi_res[site] = result[0]

raw_pi_res = pd.DataFrame(raw_pi_res, index=gbc.classes_).T
raw_pi_res

diversified-CO 52
  nsims	neutral 2374	filtering 3818	competition 3413	
  Training ML
forest-MN 85
  nsims	neutral 3141	filtering 2940	competition 2164	
  Training ML
diversified-NO 93
  nsims	neutral 3072	filtering 2700	competition 1965	
  Training ML
simplified-JO 71
  nsims	neutral 3102	filtering 3337	competition 2717	
  Training ML
diversified-KE 69
  nsims	neutral 3050	filtering 3401	competition 2765	
  Training ML
simplified-JE 70
  nsims	neutral 3069	filtering 3365	competition 2751	
  Training ML
forest-MK 88
  nsims	neutral 3125	filtering 2877	competition 2086	
  Training ML
simplified-TK 48
  nsims	neutral 2088	filtering 3870	competition 3672	
  Training ML
CPU times: user 4min 7s, sys: 7.98 s, total: 4min 15s
Wall time: 40.8 s


,competition,filtering,neutral
diversified-CO,0.111623,0.066736,0.821641
forest-MN,0.224386,0.020296,0.755318
diversified-NO,0.571387,0.007664,0.420949
simplified-JO,0.214151,0.008902,0.776947
diversified-KE,0.369872,0.005500,0.624627
simplified-JE,0.340487,0.001086,0.658427
forest-MK,0.834034,0.000222,0.165744
simplified-TK,0.724441,0.012640,0.262918


In [53]:
raw_pi_res

,competition,filtering,neutral
diversified-CO,0.075502,0.013446,0.911052
forest-MN,0.328942,0.003865,0.667193
diversified-NO,0.703014,0.006853,0.290133
simplified-JO,0.452398,0.002871,0.544731
diversified-KE,0.515783,0.010947,0.473271
simplified-JE,0.731779,0.000417,0.267804
forest-MK,0.816806,0.000000,0.183194
simplified-TK,0.812651,0.015042,0.172307


In [71]:
rf_params = {'bootstrap': True,
 'ccp_alpha': 0.0,
 'max_depth': 70,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}